<a href="https://colab.research.google.com/github/Blova-TB/OptimisationUQAC/blob/main/template/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMPL Model Colaboratory Template
[![colab.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Open In Deepnote](https://deepnote.com/buttons/launch-in-deepnote-small.svg)](https://deepnote.com/launch?url=https://github.com/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Open In Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/template/colab.ipynb) [![Powered by AMPL](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/template/colab.ipynb)](https://ampl.com)

Description: Basic notebook template for the AMPL Colab repository

Tags: amplpy, template, highlights

Notebook author: Filipe Brandão <<fdabrandao@gmail.com>>

In [60]:
# Install dependencies
%pip install -q amplpy pandas

In [61]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


## Use `%%ampl_eval` to evaluate AMPL commands and declarations

In [62]:
%%ampl_eval

set I;
set K;

param t {I,K} >= 0;
param B {K} >= 0;
param N {I,K} >= 0;
param l {K} >= 0;
param C >= 0;
param O >= 0;
param D {K} >= 0;

var x {I,K} >=0 integer;
var delta {I,K} >=0;

minimize Total_Cost: sum {i in I, k in K} (O * delta[i,k] + C * x[i,k]);

subject to Available_Employees {k in K}:
    sum {i in I} x[i,k] <= B[k];

subject to Overload_Definition {i in I, k in K}:
    N[i,k] * t[i,k] - D[k] * x[i,k] <= delta[i,k];

subject to Min_Employees_Per_Route {i in I, k in K}:
    x[i,k] >= l[k];

## Load data directly from Python data structures using [amplpy](https://amplpy.readthedocs.io/)

In [63]:
routes = ["BJ","BP","JP","PP","JJ","Oth"]
ampl.set["I"] = routes

periods = ["7","8","9","10","11","12","13","14","15","16","17","18","19","20","21"]
ampl.set["K"] = periods


t = [
    [18.905, 9.749, 11.534, 11.242, 11.922, 11.677, 10.743, 10.186, 10.437, 11.289, 13.351, 14.854, 14.301, 12.229, 15.88],
    [15.16, 8.835, 11.448, 11.579, 13.545, 12.118, 10.506, 10.277, 11.583, 12.979, 16.688, 19.532, 16.701, 13.915, 15.749],
    [16.435, 12.52, 14.888, 14.382, 15.909, 16.982, 15.569, 14.884, 14.722, 16.832, 24.028, 21.92, 17.958, 21.509, 18.056],
    [10.08, 7.409, 10.958, 9.925, 12.383, 11.449, 10.093, 9.88, 11.45, 15.746, 23.434, 21.796, 21.416, 21.545, 18.862],
    [14.135, 10.66, 12.586, 11.88, 13.479, 14.08, 12.096, 11.578, 11.836, 12.863, 18.467, 13.843, 11.704, 14.069, 12.677],
    [17.898, 10.554, 12.89, 12.267, 14.129, 14.385, 12.342, 12.21, 13.177, 15.018, 22.922, 21.318, 16.827, 19.808, 18.254]
]
ampl.param["t"] = {
    (routes[i], periods[j]): t[i][j]
    for i in range(len(routes))
    for j in range(len(periods))
}

ampl.param["B"] = [4,9,9,11,9,8,11,13,12,6,2,3,3,3,2]

N = [
    [0, 2, 4, 3, 2, 1, 3, 3, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 3, 3, 1, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 2, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1, 0],
    [0, 0, 4, 8, 4, 1, 5, 5, 2, 0, 0, 0, 0, 0, 0],
    [0, 2, 3, 4, 3, 1, 3, 3, 2, 1, 0, 0, 0, 0, 0],
    [1, 13, 25, 26, 19, 11, 21, 27, 18, 8, 3, 4, 3, 4, 2]
]
ampl.param["N"] = {
    (routes[i], periods[j]): N[i][j]
    for i in range(len(routes))
    for j in range(len(periods))
}

ampl.param["l"] = [1,7,6,9,7,5,9,12,8,4,2,3,2,2,0]

ampl.param["C"] = 20
ampl.param["O"] = 1000

ampl.param["D"] = [60]*len(periods)

## Solve with HiGHS

In [64]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1"
ampl.solve()

	presolve: constraint Available_Employees['20'] cannot hold:
		body <= 3 cannot be >= 12; difference = -9
	presolve: constraint Available_Employees['19'] cannot hold:
		body <= 3 cannot be >= 12; difference = -9
	presolve: constraint Available_Employees['18'] cannot hold:
		body <= 3 cannot be >= 18; difference = -15
	presolve: constraint Available_Employees['17'] cannot hold:
		body <= 2 cannot be >= 12; difference = -10
	presolve: constraint Available_Employees['16'] cannot hold:
		body <= 6 cannot be >= 24; difference = -18
	9 presolve messages suppressed.


## Retrieve solution as a pandas dataframe

In [69]:
x_df = ampl.var["x"].to_pandas()
delta_df = ampl.var["delta"].to_pandas()

print("Solution for x:")
print(x_df)

print("\nSolution for delta:")
print(delta_df)

Solution for x:
               x.val
index0 index1       
BJ     10          0
       11          0
       12          0
       13          0
       14          0
...              ...
PP     20          0
       21          0
       7           0
       8           0
       9           0

[90 rows x 1 columns]

Solution for delta:
               delta.val
index0 index1           
BJ     10              0
       11              0
       12              0
       13              0
       14              0
...                  ...
PP     20              0
       21              0
       7               0
       8               0
       9               0

[90 rows x 1 columns]
